# Mapping

Mappings are pre-defined configuration files that encode the logic on how to transform a specific data source into Resources that follow a template of a targeted _Type_. 

This notebook demonstrates the `DictionaryMapping` wich is based on a JSON structure that represent the target structure, and Python code that will apply desired transformations on the data source.

Note: commented lines are not implemented on the RdfModel

In [1]:
from kgforge.core import KnowledgeGraphForge

In [2]:
import getpass

In [3]:
token = getpass.getpass()

In [4]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", token=token)

## Imports

In [5]:
from kgforge.core import Resource

In [6]:
from kgforge.specializations.mappings import DictionaryMapping

## Data

In [7]:
scientists = [
    {
        "id": 123,
        "name": "Marie Curie",
        "gender": "female",
        "middle_name": "Salomea",
    },
    {
        "id": 456,
        "name": "Albert Einstein",
        "gender": "male",
        "middle_name": "(missing)",
    },
]

## Mapping data to the Knowledge Graph Schema

### basics

In [8]:
forge.template("Association")

{
    id: ""
    type: Association
    agent:
    {
        type: Person
        address:
        {
            type: PostalAddress
            postalCode: ""
            streetAddress: ""
        }
        birthDate: 9999-12-31
        deathDate: 9999-12-31
        gender:
        [
            female
            male
        ]
        givenName: ""
        name: ""
    }
}


In [9]:
mapping_simple = DictionaryMapping("""
    type: Association
    agent:
    {
        type: Person
        name: x.name
    }
""")

In [10]:
resources_simple = forge.map(scientists, mapping_simple)

In [11]:
print(resources_simple[0])

{
    type: Association
    agent:
    {
        type: Person
        name: Marie Curie
    }
}


### missing values

In [12]:
mapping_na = DictionaryMapping("""
    type: Association
    agent:
    {
        type: Person
        name: x.name
        additionalName: x.middle_name
    }
""")

In [13]:
print(forge.map(scientists[1], mapping_na))

{
    type: Association
    agent:
    {
        type: Person
        additionalName: (missing)
        name: Albert Einstein
    }
}


In [14]:
print(forge.map(scientists[1], mapping_na, na="(missing)"))

{
    type: Association
    agent:
    {
        type: Person
        name: Albert Einstein
    }
}


### multiple mappings

In [15]:
mapping_person = DictionaryMapping("""
    id: forge.format("identifier", "persons", x.id)
    type: Person
    name: x.name
""")

In [16]:
mapping_association = DictionaryMapping("""
    type: Association
    agent: forge.format("identifier", "persons", x.id)
""")

In [17]:
resources_graph = forge.map(scientists, [mapping_person, mapping_association])

In [18]:
print(resources_graph[0])

{
    id: https://kg.example.ch/persons/123
    type: Person
    name: Marie Curie
}


In [19]:
print(resources_graph[1])

{
    type: Association
    agent: https://kg.example.ch/persons/123
}


### managed mappings

In [20]:
forge.sources()

<action> not_supported
<error> NotSupportedError: RdfModel is not supporting _sources()



In [21]:
forge.mappings("scientists-database")

<action> not_supported
<error> NotSupportedError: RdfModel is not supporting _mappings()



In [22]:
# mapping = forge.mapping("Association", "scientists-database")

In [23]:
# resources = forge.map(scientists, mapping, na="(missing)")

In [24]:
# type(resources)

In [25]:
# type(resources[0])

In [26]:
# print(mapping)

In [27]:
# print(resources[0])

In [28]:
# forge.register(resources)

## Managing mappings

In [29]:
filepath = "mappings/scientists-database/DictionaryMapping/Association.hjson"

### saving

In [30]:
# mapping.save(filepath)

### tracking & sharing changes

In [31]:
# ! cd mappings

In [32]:
# ! git add Association.hjson

In [33]:
# ! git commit -m "Add Association mapping"

In [34]:
# ! git push

### loading

In [35]:
loaded = DictionaryMapping.load(filepath)

In [36]:
# loaded == mapping